# B404B Modern Issues in Finance: Computational Finance in Python
*Submission Date: June 29, 2025*

This notebook contains the code and explanations of **group 2** for the assessment in the course **B404B Modern Issues in Finance: Computational Finance in Python**.

**Members in Group 2**

* Aaron
* Liqian Huang
* Nino Maisuradze
* Xuan Yang


**Table of Contents**


1. [Data Acquisition](#setup)
2. [Definition of Signals](#I)
3. [Computation of Signals and Resulting Positions](#II)
4. [Statistics of the Strategy](#III)
5. [Graphs of Strategy](#IV)


## Data Acquisition


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import module

In [ ]:
# define tickers of stocks that are to be analyzed
tickers = [ \
    'AAPL', # Apple
    'MSFT', # Microsoft
    'AMZN', # Amazon
    '^GSPC'] # S&P500 - Benchmark

# define time span of stock price data
start_date = '2010-01-01'
end_date = '2021-12-31'

# download the data
df_prices, df_price_changes = module.download_stock_price_data(tickers, start_date, end_date)

## Definition of Signals <a id="I"></a>


In [ ]:
# DEFINE THREE TRADING SIGNALS

# DO YOUR RESEARCH IN A SEPARATE NOTEBOOK AND IMPLEMENT THE FINAL SIGNALS IN THE MODULE
# PLEASE PROVIDE THE FOLLOWING AS PART OF THE ASSESSMENT:
# - provide a reference to the related literature for each trading signal as mentioned below
# - provide your SEPARATE RESERACH NOTEBOOK in which you back your trading signals
#   and their parameters by empirical research:
#   - perform a systematic parameter search / optimization that backs your parameter selection empirically
#   - perform extensive in-sample and out-of-sample testing of your trading signals and parameters with respect to
#   -- companies / stocks
#   -- time horizons
#   - there is no example for your research notebook, you're completely free to develop it according to your research
# - provide a module.py file to re-use your code in both your assessment notebook and your research notebook

# REMEMBER THAT YOU MAY NOT USE BUILT-IN FUNCTIONS FROM OTHER LIBRARIES THAN NUMPY
# WHEN IN DOUBT - CODE A FUNCTION IN NUMPY ON YOUR OWN!
# EXAMPLE: .rolling().mean() is a built-in function in Pandas, that's why moving average is implemented in NumPy (see module.py)

# PLEASE MAKE SURE THAT YOUR SIGNAL FUNCTIONS DO NOT GENERATE
# A SELL SIGNAL WHEN THERE WAS NO BUY SIGNAL

This section details the construction and rationale behind the three composite trading signals employed in our strategy. Each signal is meticulously designed to capitalize on specific market phenomena by integrating distinct categories of technical indicators. This approach aims to enhance signal robustness, mitigate false positives, and ultimately contribute to maximizing risk-adjusted returns. For each composite signal, we elucidate its core idea, define its constituent sub-signals, provide the specific mathematical formulations of the underlying indicators, and present the strategy rationales and advantages.

### Signal 0: Trend-Following with Momentum


This strategy combines a long-term trend filter (Dual SMA Crossover) with a mid-term momentum trigger (MACD) to identify high-probability, trend-aligned trades.


#### Long-Term Trend Filter: Dual Moving Average (SMA) Crossover


This sub-signal establishes the primary market direction.

* **Indicator:** 50-day & 200-day Simple Moving Averages (SMA).
* **Logic:**
  * **Bullish Trend:** SMA50 > SMA200
  * **Bearish Trend:** SMA50 < SMA200
* **Formula:**
  > $$\text{SMA}_N(t) = \frac{1}{N} \sum_{i=0}^{N-1} P_{t-i}$$
* **Rationale:** Acts as a robust filter to ensure trades are only considered in the direction of the dominant market trend, effectively reducing counter-trend risk.


#### Mid-Term Momentum Trigger: MACD

This sub-signal provides the precise timing for entry and exit.

* **Indicator:** Moving Average Convergence Divergence (MACD) with standard (12, 26, 9) parameters.
* **Logic:**
  * **Buy Trigger:** MACD Line crosses **above** Signal Line.
  * **Sell Trigger:** MACD Line crosses **below** Signal Line.
* **Core Formulas:**
  > $$\text{MACD Line} = \text{EMA}_{12}(P_t) - \text{EMA}_{26}(P_t)$$
  > $$\text{Signal Line} = \text{EMA}_{9}(\text{MACD Line})$$
* **Rationale:** Pinpoints actionable shifts in momentum, serving as the confirmation needed to execute a trade within the established long-term trend.


#### Construct the signal

The final trading decision is based on the synergy of both sub-signals.

* **BUY Signal:** (SMA50 > SMA200) **AND** (MACD Line crosses above Signal Line)
* **SELL Signal:** (SMA50 < SMA200) **AND** (MACD Line crosses below Signal Line)
* **HOLD:** All other conditions.


**Key Advantages**

* **Reduces Whipsaws:** The trend filter effectively ignores false momentum signals that occur against the primary market direction.
* **Improves Signal Quality:** By demanding both trend alignment and momentum confirmation, the strategy focuses only on high-conviction trades, aiming for a higher win rate.

In [ ]:
### SIGNAL 0
def signal_0(series):
    return module.ma_signal(series, 250, 500)

### Signal 1: Mean Reversion & Volatility

This strategy aims to capture mean-reversion bounces from oversold conditions, using volatility stabilization as a confirmation to avoid "catching a falling knife."


#### Oversold Identifier: Relative Strength Index (RSI)

This sub-signal identifies when an asset's price has moved to an extreme, suggesting a potential reversal.

* **Indicator:** 14-day Relative Strength Index (RSI).
* **Logic:**
  * **Oversold Zone:** RSI < 30
  * **Overbought Zone:** RSI > 70
* **Core Formula:**
  > $$\text{RSI} = 100 - \frac{100}{1 + \text{RS}}, \quad \text{where RS} = \frac{\text{Average Gain}}{\text{Average Loss}}$$
* **Rationale:** Operates on the principle of mean reversion, where extreme price movements away from the average are statistically likely to correct.


#### Stabilization Trigger: Bollinger Bands


This sub-signal confirms that extreme price pressure is subsiding before a trade is placed.

* **Indicator:** Bollinger Bands (20-day SMA, 2 Standard Deviations).
* **Logic:**
  * **Buy Trigger:** Price crosses back **above** the Lower Band after trading below it.
  * **Sell Trigger:** Price crosses back **below** the Upper Band after trading above it.
* **Core Formula:**
  > $$\text{Upper/Lower Band} = \text{SMA}_{20}(P_t) \pm 2 \times \text{StdDev}_{20}(P_t)$$
* **Rationale:** Provides confirmation that volatility is stabilizing and the intense selling (or buying) pressure has momentarily paused, offering a safer entry point.


#### Combined Signal Rule

The final trading decision requires both an extreme condition and a stabilization signal.

* **BUY Signal:** (RSI < 30) **AND** (Price crosses back above Lower Bollinger Band)
* **SELL Signal:** (RSI > 70) **AND** (Price crosses back below Upper Bollinger Band)



**Key Advantages**

* **Avoids "Catching a Falling Knife":** The Bollinger Band trigger acts as a safety check, preventing buys into an asset that is still in a strong downtrend, even if the RSI is low.
* **Built-in Volatility Management:** By waiting for price to re-enter the bands, the strategy inherently avoids trading during the most chaotic phases of a price move, making it suitable for volatile assets.

In [ ]:
### SIGNAL 1
def signal_1(series):
    return module.ma_signal(series, 125, 250)

### Signal 2: Breakout & Trend Strength


This strategy aims to capture the beginning of new, powerful trends by confirming price breakouts with a measure of underlying trend strength, filtering out false signals.


#### Breakout Trigger: Donchian Channel

This sub-signal provides the initial trigger when price moves beyond its recent range.

* **Indicator:** 20-day Donchian Channel.
* **Logic:**
  * **Buy Trigger:** Price closes **above** the Upper Channel Band (a new 20-day high).
  * **Sell Trigger:** Price closes **below** the Lower Channel Band (a new 20-day low).
* **Core Formula:**
  > $$\text{Upper Band} = \text{Highest High over past 20 periods}$$
  > $$\text{Lower Band} = \text{Lowest Low over past 20 periods}$$
* **Rationale:** Provides a clear, objective signal that price has broken out of its recent trading range, indicating a potential new trend.


#### Trend Strength Confirmation: ADX

This sub-signal validates the conviction behind the breakout.

* **Indicator:** 14-period Average Directional Index (ADX).
* **Logic:**
  * **Strong Trend:** ADX > 25
  * **Weak / No Trend:** ADX < 25
* **Core Concept:**
  > ADX is derived from smoothed measures of positive (+DI) and negative (-DI) directional movement. It does not indicate direction, only strength.
* **Rationale:** Measures the *strength* of a trend, regardless of whether it is bullish or bearish. It acts as a critical filter to confirm a breakout has genuine force behind it.


#### Combined Signal Rule

The final trading decision requires both a breakout and confirmed trend strength.

* **BUY Signal:** (Price breaks above Upper Donchian Channel) **AND** (ADX > 25)
* **SELL Signal:** (Price breaks below Lower Donchian Channel) **AND** (ADX > 25)

**Key Advantages**

* **Filters "Fakeouts":** The ADX requirement helps distinguish genuine, powerful breakouts from low-conviction moves that are likely to fail and reverse.
* **Captures Early Trend Initiation:** Designed to enter at the beginning of major, sustainable moves, aiming to maximize profit from the entirety of a new trend.

In [ ]:
### SIGNAL 2
def signal_2(series):
    return module.ma_signal(series, 60, 120)

## Computation of Signals and Resulting Positions

In [ ]:
# Compute signals
signals = {
    tickers[0]: signal_0(df_prices[tickers[0]]),
    tickers[1]: signal_1(df_prices[tickers[1]]),
    tickers[2]: signal_2(df_prices[tickers[2]])}
df_position_open = pd.concat([
    signals[tickers[0]]['signal'].rename(tickers[0]),
    signals[tickers[1]]['signal'].rename(tickers[1]),
    signals[tickers[2]]['signal'].rename(tickers[2])], axis = 1)
df_position_changes = pd.concat([
    signals[tickers[0]]['position_change'].rename(tickers[0]),
    signals[tickers[1]]['position_change'].rename(tickers[1]),
    signals[tickers[2]]['position_change'].rename(tickers[2])], axis = 1)

In [ ]:
# ALLOCATE CAPITAL AND COMPUTE RESULTING POSITIONS
initial_cash = 1.0
capital_fraction_per_trade = 0.2

# DO NOT MODIFY THIS CELL BELOW THIS LINE
position = []

def open_trades(position, position_change):
    vec = np.maximum([position_change[ticker] for ticker in tickers[:-1]], [0])
    vec = position[-1] * (1 - np.power((1 - capital_fraction_per_trade), np.sum(vec))) * vec / (1 if (np.nansum(vec) == 0.0) else np.nansum(vec))
    return np.append(vec + position[:-1], position[-1] - np.sum(vec))

def hold_trades(position, price_change):
    return np.concatenate((position[:-1] * price_change[:-1], [position[-1]]))

def close_trades(position, position_change):
    vec = np.concatenate((np.array([position_change[ticker] < 0.0 for ticker in tickers[:-1]]), [False]))
    position[-1] = position[-1] + np.sum(position[vec])
    position[vec] = 0.0
    return position

is_first = True
for idx, position_change in df_position_changes.iterrows():
    if is_first:
        position.append(open_trades(np.concatenate((np.zeros(len(df_position_changes.columns)), [initial_cash])), position_change))
        is_first = False
    else:
        hlpr_pos = hold_trades(position[-1], df_price_changes.loc[[idx]].to_numpy()[0])
        hlpr_pos = close_trades(hlpr_pos, position_change)
        position.append(open_trades(hlpr_pos, position_change))

df_position = pd.DataFrame(position, index = df_prices.index, columns = tickers[:-1] + ['cash'])

## Statistics of the Strategy

Here we define the key (mostly risk-adjusted) performance measures used to evaluate the strategy.

#### **Symbol Definitions (符号定义)**

* $R_p$: Portfolio Return (投资组合回报率)
* $R_f$: Risk-Free Rate (无风险利率)
* $R_M$: Market / Benchmark Return (市场或基准回报率)
* $\beta_p$: Portfolio Beta (投资组合的贝塔系数, 系统性风险)
* $\sigma_p$: Standard Deviation of portfolio returns (投资组合回报率的标准差, 总风险)
* $\sigma_d$: Standard Deviation of downside returns (下行标准差, 下行风险)
* $\sigma_{pm}$: Tracking Error, the standard deviation of the difference between portfolio and market returns, i.e., $StdDev(R_p - R_M)$.
* $\text{MaxDD}$: Maximum Drawdown (最大回撤)



#### **1. Core Risk-Adjusted Return Ratios**
*This group of ratios measures the return generated per unit of risk taken. It measures the "bang for your buck": for each unit of risk taken, how much return is generated?*

* **Sharpe Ratio**: Measures the excess return of an investment per unit of its total volatility (standard deviation).
Answers the question: "For every unit of total risk (both good and bad volatility) I take, how much excess return do I get back?" It is the most common measure of an investment's "bang for your buck".
 > $\text{Sharpe Ratio} = \frac{R_p - R_f}{\sigma_p}$

* **Sortino Ratio**: A "smarter" variation of the Sharpe Ratio that measures excess return per unit of downside risk (harmful volatility).
It only considers "bad" risk (when prices go down), answering: "For every unit of *losing* risk I take, how much reward am I getting?"
 > $\text{Sortino Ratio} = \frac{R_p - R_f}{\sigma_d}$

* **Treynor Ratio**: Measures the excess return earned per unit of systematic risk, as defined by beta (β).
It answers: "For every unit of market risk I took on, how much reward did I get?" This is especially useful for judging a single stock within a diversified portfolio.

 > $\text{Treynor Ratio} = \frac{R_p - R_f}{\beta_p}$



#### **2. Performance Relative to a Benchmark**
*This group measures the strategy's ability to outperform a given market benchmark, assessing manager skill. It measures "true skill": is the strategy's success due to luck or genuine expertise?*


* **Jensen's Alpha (α)**: Measures the portfolio's abnormal return over the theoretical expected return predicted by the Capital Asset Pricing Model (CAPM). This is the measure of a manager's or strategy's "true skill". It calculates the return earned above and beyond what was expected, given the market's performance and the risk taken. A positive Alpha suggests genuine skill.

 > $\alpha_J = R_p - [R_f + \beta_p(R_M - R_f)]$

* **Information Ratio (IR)**: Measures the *consistency* of a portfolio's excess returns over a benchmark relative to the volatility of those returns (tracking error). It asks: "How consistently does my strategy beat the benchmark, and is that outperformance smooth or erratic?" A high IR indicates that the ability to outperform is reliable.

 > $\text{Information Ratio} = \frac{R_p - R_M}{\sigma_{pm}}$



#### **3. Risk & Drawdown-Based Measures**
*This group focuses on performance during adverse periods and quantifies potential losses. It focuses on "resilience": how does the strategy perform under stress and during its worst periods?*
*

* **Value-at-Risk (VaR)**: A statistical measure that estimates the maximum potential loss over a specific time frame for a given confidence level. It's the "sleep-at-night" number. It answers a simple question: "What is the most I can expect to lose over a given period (e.g., one day) with 95% confidence?" It is a key metric for risk management.


 > *Note: VaR is a direct risk value, not a ratio calculated from the symbols above.*

* **Calmar Ratio**: Measures risk-adjusted return by dividing the annualized rate of return by the absolute value of the portfolio's maximum drawdown. It's a "pain vs. gain" ratio popular with professional traders. It compares the annual return to the worst loss (maximum drawdown) the strategy ever experienced. A high Calmar ratio means the strategy recovers well from its worst periods.

 > $\text{Calmar Ratio} = \frac{R_p}{\vert\text{MaxDD}\vert}$



#### **4. Ratios for Easier Interpretation**
*This metric adjusts performance to make it more directly comparable with a benchmark.*

* **Modigliani Ratio (M² Ratio)**: An "apples-to-apples" comparison tool. It adjusts the portfolio's risk to be exactly the same as a benchmark's, then shows what the portfolio's return *would have been*. This makes it incredibly easy to see if we truly beat the market on a risk-adjusted basis.

 > $M^2 = (\frac{R_p - R_f}{\sigma_p}) \cdot \sigma_M + R_f$

In [ ]:
# COMPUTE MEANINGFUL STATISTICS OF YOUR STRATEGY
# YOU ARE FREE TO CHOOSE MEASURES

# REMEMBER THAT YOU MAY NOT USE READY-TO-USE FUNCTIONS FROM OTHER LIBRARIES THAN NUMPY
# WHEN IN DOUBT - CODE A FUNCTION ON YOUR OWN!
# EXAMPLE: .mean() and .std() are ready-to-use, that's why they are implemented using NumPy below

returns = df_position.sum(axis=1)
returns = (returns[1:].to_numpy() / returns[:-1].to_numpy()) - 1
mean_returns = np.sum(returns) / len(returns)
std_returns = np.sqrt(np.sum(np.square(returns - mean_returns)) / len(returns))
print('Annualized mean: ' + str(mean_returns * 250))
print('Annualized std:  ' + str(std_returns * np.sqrt(250)))

## Graphs of Strategy

In [ ]:
# COMPUTE MEANINGFUL PLOTS OF YOUR STRATEGY AND LABEL THEM IN AN UNDERSTANDABLE WAY
df_position.sum(axis=1).plot()